In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from torch import autocast
from torch.cuda.amp import GradScaler

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Utiliy Functions.
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [3]:
# Ensure that we get deterministic results.
SEED=1
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [4]:
# Setup TensorBoard
writer = SummaryWriter(log_dir="/data/logs")

In [5]:
# Check that we have a CUDA enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
MEAN_RGB = [0.47889522, 0.47227842, 0.43047404]
STD_RGB = [0.229, 0.224, 0.225]
IMG_SIZE = 64

In [7]:
# Load train and val dataset. 
TRAINDIR = "/data/train" # ImageNet train.
VALDIR = "/data/val" # ImageNet val.
BATCH_SIZE = 64
TRAIN_WORKERS = 6
VAL_WORKERS = 0
SHUFFLE = False

# Scale to ImageNet mean and STD since we will be using a model pretrained on ImageNet. 
transform_train = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(MEAN_RGB, STD_RGB),
])

transform_val = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(MEAN_RGB, STD_RGB),
])

# Load training data.
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=BATCH_SIZE, num_workers=TRAIN_WORKERS, shuffle=SHUFFLE, pin_memory=True, sampler=None)

# Load validation data.
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=BATCH_SIZE, num_workers=VAL_WORKERS, shuffle=SHUFFLE, pin_memory=True, sampler=None)

In [9]:
NUM_CLASSES = 1000
ARCH = 'resnet18'
LR = 1e-4
# Load model from PyTorch.
model = models.__dict__[ARCH]()
inf = model.fc.in_features
# Set fully connected layer to train with 1000 classes.
model.fc = nn.Linear(inf, NUM_CLASSES)
model.to(device)
# Setup optimizer and loss function. 
optimizer = torch.optim.Adam(model.parameters(), LR)
criterion = nn.CrossEntropyLoss().to(device)

In [10]:
PRINT_FREQ = BATCH_SIZE - 1 
global_step = 0

In [12]:
scaler = GradScaler()
# Define train step.
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step
    # Keep progress of metrics.
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # Switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # Measure data loading time.
        data_time.update(time.time() - end)
        
        # Move data to device
        images = images.to(device, non_blocking=True)
        target = target.to(device, non_blocking=True)
            
        # Use automatic mixed precision (AMP) to increase training speed.
        with autocast("cuda"):
            # Compute output.
            output = model(images)
            loss = criterion(output, target)

        # Measure accuracy and record loss.
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))
        
        # Write tensorboard logs.
        writer.add_scalar("Loss/train", loss, global_step=global_step)
        writer.add_scalar("Acc1/train", top1.avg, global_step=global_step)
        writer.add_scalar("Acc5/train", top5.avg, global_step=global_step)
        global_step += 1

        # Compute gradient.
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        # Measure elapsed time.
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [13]:
# Define validation step.
def validate(val_loader, model, criterion):
    global global_step
    # Keep progress of metrics.
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # Switch to evaluate mode.
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            
            # Move data to GPU if CUDA device is available.
            images = images.to(device, non_blocking=True)
            target = target.to(device, non_blocking=True)

            # Compute output.
            output = model(images)
            loss = criterion(output, target)

            # Measure accuracy and record loss.
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))
            

            # Measure elapsed time.
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
    
     # Write tensorboard logs.
    writer.add_scalar("Loss/val", loss, global_step=global_step)
    writer.add_scalar("Acc1/val", top1.avg, global_step=global_step)
    writer.add_scalar("Acc5/val", top5.avg, global_step=global_step)
    
    return top1.avg

In [14]:
for epoch in range(1):
    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    validate(val_loader, model, criterion)


Epoch: [0][    0/20019]	Time  1.070 ( 1.070)	Data  0.482 ( 0.482)	Loss 6.6420e+00 (6.6420e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Epoch: [0][   63/20019]	Time  0.171 ( 0.065)	Data  0.143 ( 0.030)	Loss 7.8548e+00 (4.9690e+00)	Acc@1   0.00 ( 23.88)	Acc@5   0.00 ( 62.82)
Epoch: [0][  126/20019]	Time  0.027 ( 0.060)	Data  0.000 ( 0.030)	Loss 6.4669e+00 (5.0832e+00)	Acc@1   0.00 ( 13.25)	Acc@5   0.00 ( 53.72)
Epoch: [0][  189/20019]	Time  0.205 ( 0.061)	Data  0.179 ( 0.032)	Loss 8.1459e+00 (4.9340e+00)	Acc@1   0.00 ( 14.89)	Acc@5   0.00 ( 48.04)
Epoch: [0][  252/20019]	Time  0.026 ( 0.060)	Data  0.000 ( 0.031)	Loss 5.6542e+00 (5.1968e+00)	Acc@1   0.00 ( 11.18)	Acc@5   0.00 ( 36.90)
Epoch: [0][  315/20019]	Time  0.266 ( 0.060)	Data  0.240 ( 0.032)	Loss 5.9663e+00 (5.2559e+00)	Acc@1   0.00 (  9.61)	Acc@5   0.00 ( 31.48)
Epoch: [0][  378/20019]	Time  0.026 ( 0.061)	Data  0.000 ( 0.033)	Loss 4.5926e+00 (5.3821e+00)	Acc@1   0.00 (  8.01)	Acc@5   0.00 ( 26.29)
Epoch: [0][  441/20019]	Tim

Epoch: [0][ 3717/20019]	Time  0.028 ( 0.062)	Data  0.000 ( 0.036)	Loss 8.6962e+00 (7.8893e+00)	Acc@1   0.00 (  0.87)	Acc@5   0.00 (  2.87)
Epoch: [0][ 3780/20019]	Time  0.025 ( 0.062)	Data  0.000 ( 0.035)	Loss 8.6346e+00 (7.9001e+00)	Acc@1   0.00 (  0.86)	Acc@5   0.00 (  2.82)
Epoch: [0][ 3843/20019]	Time  0.028 ( 0.061)	Data  0.000 ( 0.035)	Loss 8.6217e+00 (7.9131e+00)	Acc@1   0.00 (  0.85)	Acc@5   0.00 (  2.78)
Epoch: [0][ 3906/20019]	Time  0.053 ( 0.061)	Data  0.029 ( 0.035)	Loss 8.4636e+00 (7.9249e+00)	Acc@1   0.00 (  0.83)	Acc@5   0.00 (  2.73)
Epoch: [0][ 3969/20019]	Time  0.026 ( 0.061)	Data  0.000 ( 0.035)	Loss 9.3258e+00 (7.9425e+00)	Acc@1   0.00 (  0.82)	Acc@5   0.00 (  2.69)
Epoch: [0][ 4032/20019]	Time  0.074 ( 0.061)	Data  0.048 ( 0.035)	Loss 8.6697e+00 (7.9541e+00)	Acc@1   0.00 (  0.81)	Acc@5   0.00 (  2.65)
Epoch: [0][ 4095/20019]	Time  0.027 ( 0.061)	Data  0.001 ( 0.035)	Loss 8.0108e+00 (7.9629e+00)	Acc@1   0.00 (  0.79)	Acc@5   0.00 (  2.61)
Epoch: [0][ 4158/20019]	Tim

Epoch: [0][ 7434/20019]	Time  0.026 ( 0.060)	Data  0.000 ( 0.034)	Loss 7.1484e+00 (7.6373e+00)	Acc@1   0.00 (  0.44)	Acc@5   0.00 (  1.44)
Epoch: [0][ 7497/20019]	Time  0.026 ( 0.060)	Data  0.000 ( 0.034)	Loss 7.1094e+00 (7.6328e+00)	Acc@1   0.00 (  0.43)	Acc@5   0.00 (  1.42)
Epoch: [0][ 7560/20019]	Time  0.030 ( 0.060)	Data  0.000 ( 0.034)	Loss 7.0859e+00 (7.6285e+00)	Acc@1   0.00 (  0.43)	Acc@5   0.00 (  1.41)
Epoch: [0][ 7623/20019]	Time  0.026 ( 0.060)	Data  0.000 ( 0.034)	Loss 7.1680e+00 (7.6240e+00)	Acc@1   0.00 (  0.43)	Acc@5   0.00 (  1.40)
Epoch: [0][ 7686/20019]	Time  0.027 ( 0.060)	Data  0.000 ( 0.034)	Loss 7.0977e+00 (7.6198e+00)	Acc@1   0.00 (  0.42)	Acc@5   0.00 (  1.39)
Epoch: [0][ 7749/20019]	Time  0.172 ( 0.060)	Data  0.147 ( 0.034)	Loss 7.1484e+00 (7.6155e+00)	Acc@1   0.00 (  0.42)	Acc@5   0.00 (  1.38)
Epoch: [0][ 7812/20019]	Time  0.073 ( 0.060)	Data  0.046 ( 0.034)	Loss 7.1250e+00 (7.6116e+00)	Acc@1   0.00 (  0.42)	Acc@5   0.00 (  1.37)
Epoch: [0][ 7875/20019]	Tim

Epoch: [0][11151/20019]	Time  0.025 ( 0.059)	Data  0.000 ( 0.033)	Loss 7.2933e+00 (7.4852e+00)	Acc@1   0.00 (  0.29)	Acc@5   0.00 (  0.96)
Epoch: [0][11214/20019]	Time  0.159 ( 0.059)	Data  0.133 ( 0.033)	Loss 7.2773e+00 (7.4840e+00)	Acc@1   0.00 (  0.29)	Acc@5   0.00 (  0.95)
Epoch: [0][11277/20019]	Time  0.026 ( 0.059)	Data  0.000 ( 0.033)	Loss 7.5685e+00 (7.4829e+00)	Acc@1   0.00 (  0.29)	Acc@5   0.00 (  0.95)
Epoch: [0][11340/20019]	Time  0.026 ( 0.059)	Data  0.000 ( 0.033)	Loss 7.3477e+00 (7.4820e+00)	Acc@1   0.00 (  0.29)	Acc@5   0.00 (  0.94)
Epoch: [0][11403/20019]	Time  0.026 ( 0.059)	Data  0.000 ( 0.033)	Loss 7.2426e+00 (7.4809e+00)	Acc@1   0.00 (  0.28)	Acc@5   0.00 (  0.94)
Epoch: [0][11466/20019]	Time  0.027 ( 0.059)	Data  0.000 ( 0.033)	Loss 7.2461e+00 (7.4798e+00)	Acc@1   0.00 (  0.28)	Acc@5   0.00 (  0.93)
Epoch: [0][11529/20019]	Time  0.032 ( 0.059)	Data  0.000 ( 0.033)	Loss 7.2305e+00 (7.4786e+00)	Acc@1   0.00 (  0.28)	Acc@5   0.00 (  0.93)
Epoch: [0][11592/20019]	Tim

Epoch: [0][14868/20019]	Time  0.026 ( 0.059)	Data  0.000 ( 0.032)	Loss 7.4453e+00 (7.4455e+00)	Acc@1   0.00 (  0.22)	Acc@5   0.00 (  0.72)
Epoch: [0][14931/20019]	Time  0.027 ( 0.059)	Data  0.000 ( 0.032)	Loss 7.4336e+00 (7.4454e+00)	Acc@1   0.00 (  0.22)	Acc@5   0.00 (  0.71)
Epoch: [0][14994/20019]	Time  0.183 ( 0.058)	Data  0.156 ( 0.032)	Loss 7.4453e+00 (7.4451e+00)	Acc@1   0.00 (  0.22)	Acc@5   0.00 (  0.71)
Epoch: [0][15057/20019]	Time  0.093 ( 0.059)	Data  0.066 ( 0.032)	Loss 7.4609e+00 (7.4450e+00)	Acc@1   0.00 (  0.22)	Acc@5   0.00 (  0.71)
Epoch: [0][15120/20019]	Time  0.026 ( 0.058)	Data  0.000 ( 0.032)	Loss 7.3750e+00 (7.4449e+00)	Acc@1   0.00 (  0.21)	Acc@5   0.00 (  0.71)
Epoch: [0][15183/20019]	Time  0.319 ( 0.059)	Data  0.267 ( 0.032)	Loss 7.4648e+00 (7.4447e+00)	Acc@1   0.00 (  0.21)	Acc@5   0.00 (  0.70)
Epoch: [0][15246/20019]	Time  0.026 ( 0.058)	Data  0.000 ( 0.032)	Loss 7.3438e+00 (7.4444e+00)	Acc@1   0.00 (  0.21)	Acc@5   0.00 (  0.70)
Epoch: [0][15309/20019]	Tim

Epoch: [0][18585/20019]	Time  0.202 ( 0.058)	Data  0.176 ( 0.032)	Loss 7.5742e+00 (7.4484e+00)	Acc@1   0.00 (  0.17)	Acc@5   0.00 (  0.57)
Epoch: [0][18648/20019]	Time  0.027 ( 0.058)	Data  0.000 ( 0.032)	Loss 7.5586e+00 (7.4488e+00)	Acc@1   0.00 (  0.17)	Acc@5   0.00 (  0.57)
Epoch: [0][18711/20019]	Time  0.213 ( 0.058)	Data  0.186 ( 0.032)	Loss 7.5938e+00 (7.4491e+00)	Acc@1   0.00 (  0.17)	Acc@5   0.00 (  0.57)
Epoch: [0][18774/20019]	Time  0.026 ( 0.058)	Data  0.001 ( 0.032)	Loss 7.5469e+00 (7.4495e+00)	Acc@1   0.00 (  0.17)	Acc@5   0.00 (  0.57)
Epoch: [0][18837/20019]	Time  0.224 ( 0.058)	Data  0.197 ( 0.032)	Loss 7.5234e+00 (7.4496e+00)	Acc@1   0.00 (  0.17)	Acc@5   0.00 (  0.57)
Epoch: [0][18900/20019]	Time  0.027 ( 0.058)	Data  0.000 ( 0.032)	Loss 7.5859e+00 (7.4500e+00)	Acc@1   0.00 (  0.17)	Acc@5   0.00 (  0.56)
Epoch: [0][18963/20019]	Time  0.026 ( 0.058)	Data  0.000 ( 0.032)	Loss 7.5409e+00 (7.4504e+00)	Acc@1   0.00 (  0.17)	Acc@5   0.00 (  0.56)
Epoch: [0][19026/20019]	Tim